In [ ]:
!pip install anfis

     |████████████████████████████████| 993 kB 3.9 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=d666ac36249194b42d10c08762a70b86e78776ba505be80c6ce1cf51772f9867
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 03 07:30:34 2014

@author: tim.meggs
"""
import itertools
import numpy as np
from membership import mfDerivs
import copy

class ANFIS:
    """Class to implement an Adaptive Network Fuzzy Inference System: ANFIS"

    Attributes:
        X
        Y
        XLen
        memClass
        memFuncs
        memFuncsByVariable
        rules
        consequents
        errors
        memFuncsHomo
        trainingType


    """

    def __init__(self, X, Y, memFunction):
        self.X = np.array(copy.copy(X))
        self.Y = np.array(copy.copy(Y))
        self.XLen = len(self.X)
        self.memClass = copy.deepcopy(memFunction)
        self.memFuncs = self.memClass.MFList
        self.memFuncsByVariable = [[x for x in range(len(self.memFuncs[z]))] for z in range(len(self.memFuncs))]
        self.rules = np.array(list(itertools.product(*self.memFuncsByVariable)))
        self.consequents = np.empty(self.Y.ndim * len(self.rules) * (self.X.shape[1] + 1))
        self.consequents.fill(0)
        self.errors = np.empty(0)
        self.memFuncsHomo = all(len(i)==len(self.memFuncsByVariable[0]) for i in self.memFuncsByVariable)
        self.trainingType = 'Not trained yet'

    def LSE(self, A, B, initialGamma = 1000.):
        coeffMat = A
        rhsMat = B
        S = np.eye(coeffMat.shape[1])*initialGamma
        x = np.zeros((coeffMat.shape[1],1)) # need to correct for multi-dim B
        for i in range(len(coeffMat[:,0])):
            a = coeffMat[i,:]
            b = np.array(rhsMat[i])
            S = S - (np.array(np.dot(np.dot(np.dot(S,np.matrix(a).transpose()),np.matrix(a)),S)))/(1+(np.dot(np.dot(S,a),a)))
            x = x + (np.dot(S,np.dot(np.matrix(a).transpose(),(np.matrix(b)-np.dot(np.matrix(a),x)))))
        return x

    def trainHybridJangOffLine(self, epochs=5, tolerance=1e-5, initialGamma=1000, k=0.01):

        self.trainingType = 'trainHybridJangOffLine'
        convergence = False
        epoch = 1

        while (epoch < epochs) and (convergence is not True):

            #layer four: forward pass
            [layerFour, wSum, w] = forwardHalfPass(self, self.X)

            #layer five: least squares estimate
            layerFive = np.array(self.LSE(layerFour,self.Y,initialGamma))
            self.consequents = layerFive
            layerFive = np.dot(layerFour,layerFive)

            #error
            error = np.sum((self.Y-layerFive.T)**2)
            #print('current error: '+ str(error))
            average_error = np.average(np.absolute(self.Y-layerFive.T))
            self.errors = np.append(self.errors,error)

            if len(self.errors) != 0:
                if self.errors[len(self.errors)-1] < tolerance:
                    convergence = True

            # back propagation
            if convergence is not True:
                cols = range(len(self.X[0,:]))
                dE_dAlpha = list(backprop(self, colX, cols, wSum, w, layerFive) for colX in range(self.X.shape[1]))


            if len(self.errors) >= 4:
                if (self.errors[-4] > self.errors[-3] > self.errors[-2] > self.errors[-1]):
                    k = k * 1.1

            if len(self.errors) >= 5:
                if (self.errors[-1] < self.errors[-2]) and (self.errors[-3] < self.errors[-2]) and (self.errors[-3] < self.errors[-4]) and (self.errors[-5] > self.errors[-4]):
                    k = k * 0.9

            ## handling of variables with a different number of MFs
            t = []
            for x in range(len(dE_dAlpha)):
                for y in range(len(dE_dAlpha[x])):
                    for z in range(len(dE_dAlpha[x][y])):
                        t.append(dE_dAlpha[x][y][z])

            eta = k / np.abs(np.sum(t))

            if(np.isinf(eta)):
                eta = k

            ## handling of variables with a different number of MFs
            dAlpha = copy.deepcopy(dE_dAlpha)
            if not(self.memFuncsHomo):
                for x in range(len(dE_dAlpha)):
                    for y in range(len(dE_dAlpha[x])):
                        for z in range(len(dE_dAlpha[x][y])):
                            dAlpha[x][y][z] = -eta * dE_dAlpha[x][y][z]
            else:
                dAlpha = -eta * np.array(dE_dAlpha)


            for varsWithMemFuncs in range(len(self.memFuncs)):
                for MFs in range(len(self.memFuncsByVariable[varsWithMemFuncs])):
                    paramList = sorted(self.memFuncs[varsWithMemFuncs][MFs][1])
                    for param in range(len(paramList)):
                        self.memFuncs[varsWithMemFuncs][MFs][1][paramList[param]] = self.memFuncs[varsWithMemFuncs][MFs][1][paramList[param]] + dAlpha[varsWithMemFuncs][MFs][param]
            epoch = epoch + 1


        self.fittedValues = predict(self,self.X)
        self.residuals = self.Y - self.fittedValues[:,0]

        return self.fittedValues



    

def forwardHalfPass(ANFISObj, Xs):
    layerFour = np.empty(0,)
    wSum = []

    for pattern in range(len(Xs[:,0])):
        #layer one
        layerOne = ANFISObj.memClass.evaluateMF(Xs[pattern,:])

        #layer two
        miAlloc = [[layerOne[x][ANFISObj.rules[row][x]] for x in range(len(ANFISObj.rules[0]))] for row in range(len(ANFISObj.rules))]
        layerTwo = np.array([np.product(x) for x in miAlloc]).T
        if pattern == 0:
            w = layerTwo
        else:
            w = np.vstack((w,layerTwo))

        #layer three
        wSum.append(np.sum(layerTwo))
        if pattern == 0:
            wNormalized = layerTwo/wSum[pattern]
        else:
            wNormalized = np.vstack((wNormalized,layerTwo/wSum[pattern]))

        #prep for layer four (bit of a hack)
        layerThree = layerTwo/wSum[pattern]
        rowHolder = np.concatenate([x*np.append(Xs[pattern,:],1) for x in layerThree])
        layerFour = np.append(layerFour,rowHolder)

    w = w.T
    wNormalized = wNormalized.T

    layerFour = np.array(np.array_split(layerFour,pattern + 1))

    return layerFour, wSum, w


def backprop(ANFISObj, columnX, columns, theWSum, theW, theLayerFive):

    paramGrp = [0]* len(ANFISObj.memFuncs[columnX])
    for MF in range(len(ANFISObj.memFuncs[columnX])):

        parameters = np.empty(len(ANFISObj.memFuncs[columnX][MF][1]))
        timesThru = 0
        for alpha in sorted(ANFISObj.memFuncs[columnX][MF][1].keys()):

            bucket3 = np.empty(len(ANFISObj.X))
            for rowX in range(len(ANFISObj.X)):
                varToTest = ANFISObj.X[rowX,columnX]
                tmpRow = np.empty(len(ANFISObj.memFuncs))
                tmpRow.fill(varToTest)

                bucket2 = np.empty(ANFISObj.Y.ndim)
                for colY in range(ANFISObj.Y.ndim):

                    rulesWithAlpha = np.array(np.where(ANFISObj.rules[:,columnX]==MF))[0]
                    adjCols = np.delete(columns,columnX)

                    senSit = mfDerivs.partial_dMF(ANFISObj.X[rowX,columnX],ANFISObj.memFuncs[columnX][MF],alpha)
                    # produces d_ruleOutput/d_parameterWithinMF
                    dW_dAplha = senSit * np.array([np.prod([ANFISObj.memClass.evaluateMF(tmpRow)[c][ANFISObj.rules[r][c]] for c in adjCols]) for r in rulesWithAlpha])

                    bucket1 = np.empty(len(ANFISObj.rules[:,0]))
                    for consequent in range(len(ANFISObj.rules[:,0])):
                        fConsequent = np.dot(np.append(ANFISObj.X[rowX,:],1.),ANFISObj.consequents[((ANFISObj.X.shape[1] + 1) * consequent):(((ANFISObj.X.shape[1] + 1) * consequent) + (ANFISObj.X.shape[1] + 1)),colY])
                        acum = 0
                        if consequent in rulesWithAlpha:
                            acum = dW_dAplha[np.where(rulesWithAlpha==consequent)] * theWSum[rowX]

                        acum = acum - theW[consequent,rowX] * np.sum(dW_dAplha)
                        acum = acum / theWSum[rowX]**2
                        bucket1[consequent] = fConsequent * acum

                    sum1 = np.sum(bucket1)

                    if ANFISObj.Y.ndim == 1:
                        bucket2[colY] = sum1 * (ANFISObj.Y[rowX]-theLayerFive[rowX,colY])*(-2)
                    else:
                        bucket2[colY] = sum1 * (ANFISObj.Y[rowX,colY]-theLayerFive[rowX,colY])*(-2)

                sum2 = np.sum(bucket2)
                bucket3[rowX] = sum2

            sum3 = np.sum(bucket3)
            parameters[timesThru] = sum3
            timesThru = timesThru + 1

        paramGrp[MF] = parameters

    return paramGrp


def predict(ANFISObj, varsToTest):

    [layerFour, wSum, w] = forwardHalfPass(ANFISObj, varsToTest)

    #layer five
    layerFive = np.dot(layerFour,ANFISObj.consequents)

    return layerFive


if __name__ == "__main__":
    print("I am main!")

I am main!


In [ ]:
import pandas as pd
df = pd.read_csv('small_data.csv')


In [ ]:
label = df.iloc[:,-1].tolist()
print(label)

[0, 0, 0, 0, 0, 1, 0, 0, 1, 1]


In [ ]:
import membership.mfDerivs
import membership.membershipfunction
import numpy

mf = [[['gaussmf',{'mean':0.,'sigma':1.}],['gaussmf',{'mean':-1.,'sigma':2.}],['gaussmf',{'mean':-4.,'sigma':10.}],['gaussmf',{'mean':-7.,'sigma':7.}]],
            [['gaussmf',{'mean':1.,'sigma':2.}],['gaussmf',{'mean':2.,'sigma':3.}],['gaussmf',{'mean':-2.,'sigma':10.}],['gaussmf',{'mean':-10.5,'sigma':5.}]]]

mfc = membership.membershipfunction.MemFuncs(mf)

In [ ]:
print(len(df.columns))

128


In [ ]:
import math 
import warnings
warnings.filterwarnings("ignore")
th=0.4
anfisvalues=[]

newdf = pd.DataFrame()
i=1
j=1
for column in df:
  while(j<127):
    f1=df.iloc[:,i].tolist()
    f2=df.iloc[:,j].tolist()
    f1_f2=np.column_stack((f1,f2))
    anf = ANFIS(f1_f2, label, mfc)
    anf.trainHybridJangOffLine(epochs=5)
    res=round(anf.fittedValues[9][0],6)
    if math.isnan(res):
      res=0
    anfisvalues.append(res)
    j=j+1
  print("index of i " + str(i))
  print(anfisvalues)
  avganfisval= sum(anfisvalues)/len(anfisvalues)
  print("avg value of anfis for feature " ,str(i), str(avganfisval))  
  if avganfisval>th :
     newdf["feature" + str(i)]=f1 
  anfisvalues.clear()   
  i=i+1
  j=1
  if i==127 :
    break
    


index of i 1
[0.427476, 0.499058, 0.497028, 0.922825, 0.93229, 0.427116, 0.979924, 0.992086, 0.383534, 0.536716, 0.592569, 0.427545, 0.427532, 0.360935, 0.733078, 0.552526, 0.47622, 0.644065, 0, 0.342665, 0.59199, 0.486821, 0.427116, 0.401496, 0.497028, 0.427116, 0.489954, 0.333338, 0.995256, 0.485158, 0.486845, 0.427116, 0.401496, 0.497028, 0.427116, 0.489954, 0.333338, 0.995256, 0.485158, 0.577152, 0.427532, 0.406879, 0.498103, 0.427532, 0.467863, 0.334082, 1.00062, 0.497948, 0.427545, 0.427545, 0.427545, 0.427545, 0.427545, 0.427545, 0.427545, 0.427545, 0.427545, 0.490112, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.427116, 0.42711

In [ ]:
newdf["label"]=label
newdf.head(10)

,feature1,feature2,feature4,feature5,feature7,feature8,feature11,feature15,feature16,feature18,...,feature47,feature48,feature58,feature103,feature107,feature111,feature114,feature118,feature121,label
0,0,1,1,0,1,0,11,37.2,84,7.00000,...,1,1,0,1,0,0,0,0,0,0
1,1,0,1,0,1,0,12,40.6,105,2.70000,...,1,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,8,38.7,68,2.59956,...,1,-2,0,1,1,0,0,1,1,0
3,0,1,1,0,1,0,16,37.4,94,5.10000,...,1,1,1,1,1,0,0,0,0,0
4,1,0,0,1,1,0,10,36.7,69,2.30000,...,1,0,0,0,0,0,0,0,0,0
5,0,1,0,1,1,0,8,35.0,71,1.00000,...,1,1,1,1,1,0,0,0,0,1
6,1,0,1,0,1,0,14,36.1,77,1.30033,...,1,0,1,1,1,0,0,1,0,0
7,0,1,1,0,1,0,12,39.0,60,3.70000,...,1,1,1,1,1,0,0,0,1,0
8,0,1,1,0,1,0,9,37.3,82,4.59956,...,0,1,0,1,1,0,1,1,0,1
9,0,1,0,1,0,1,9,35.2,60,1.70000,...,-6,1,0,0,1,1,1,1,0,1


In [ ]:
#dividing the X and the Y from the dataset
#data is dataframe after selecting feature
X=newdf.drop(['label'], axis=1)
Y=newdf.label
print(X.shape)
print(Y.shape)


# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

#basic decision tree 
from sklearn.tree import DecisionTreeClassifier
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,Y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


#Evaluating the classifier                                   # evaluation for basic decision tree
#printing every score of the classifier
#scoring in any thing
from sklearn.metrics import classification_report, accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef

print(' model DecisionTreeClassifier classifier')
acc= accuracy_score(Y_test,y_pred)
print('The accuracy is {}'.format(acc))



(10, 27)
(10,)
(7, 27)
(3, 27)
(7,)
(3,)
 model DecisionTreeClassifier classifier
The accuracy is 0.6666666666666666
